In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle
from glob import glob
from concurrent.futures import ProcessPoolExecutor, as_completed

import numpy as np
import pandas as pd
from scipy import stats

import settings as conf
from utils import is_number, chunker

# Load S-PrediXcan results

## From Rapid GWAS project

In [3]:
from results.spredixcan import PhenoResults

In [4]:
_path = os.path.join(conf.SPREDIXCAN_RESULTS_DIR['RapidGWASProject'] + '/*')
display(_path)
all_spredixcan_results_dirs = glob(_path)
display(len(all_spredixcan_results_dirs))
assert len(all_spredixcan_results_dirs) == conf.SPREDIXCAN_EXPECTED_PHENOTYPES['RapidGWASProject']

'/mnt/phenomexcan_base/results/spredixcan/rapid_gwas_project/*'

4049

In [5]:
all_spredixcan_phenotypes = [PhenoResults(p) for p in all_spredixcan_results_dirs]

display(len(all_spredixcan_phenotypes))
assert len(all_spredixcan_phenotypes) == conf.SPREDIXCAN_EXPECTED_PHENOTYPES['RapidGWASProject']

4049

## From GTEx GWAS manuscript

In [6]:
_path = os.path.join(conf.SPREDIXCAN_RESULTS_DIR['GTEX_GWAS'] + '/*')
display(_path)
all_extra_results_dirs = glob(_path)
display(len(all_extra_results_dirs))
assert len(all_extra_results_dirs) == conf.SPREDIXCAN_EXPECTED_PHENOTYPES['GTEX_GWAS']

'/mnt/phenomexcan_base/results/spredixcan/gtex_gwas/*'

42

In [7]:
all_extra_results_dirs[:5]

['/mnt/phenomexcan_base/results/spredixcan/gtex_gwas/PGC_ADHD_EUR_2017',
 '/mnt/phenomexcan_base/results/spredixcan/gtex_gwas/CNCR_Insomnia_all',
 '/mnt/phenomexcan_base/results/spredixcan/gtex_gwas/IMMUNOBASE_Systemic_lupus_erythematosus_hg19',
 '/mnt/phenomexcan_base/results/spredixcan/gtex_gwas/Astle_et_al_2016_Reticulocyte_count',
 '/mnt/phenomexcan_base/results/spredixcan/gtex_gwas/Astle_et_al_2016_Granulocyte_count']

In [8]:
_file_pattern = 'spredixcan_igwas_gtexmashrv8_(?P<code>[^/]+)__PM__(?P<tissue>.+)\.csv$'
all_extra_phenotypes = [PhenoResults(p, _file_pattern) for p in all_extra_results_dirs]
all_extra_phenotypes_plain_names = pd.Index([p.pheno_info.get_plain_name() for p in all_extra_phenotypes])

display(len(all_extra_phenotypes))
assert len(all_extra_phenotypes) == conf.SMULTIXCAN_EXPECTED_PHENOTYPES['GTEX_GWAS']

42

# S-PrediXcan: pvalues

In [9]:
from results.spredixcan import PhenoResults
from results.gtex_model import GTEXModel

In [10]:
OUTPUT_FOLDER = os.path.join(conf.GENE_ASSOC_DIR, f'spredixcan')
display(OUTPUT_FOLDER)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

'/mnt/phenomexcan_base/gene_assoc/spredixcan'

In [11]:
OUTPUT_FILE_FORMAT = 'spredixcan-{tissue}-{column}.{format}'
display(OUTPUT_FILE_FORMAT)

'spredixcan-{tissue}-{column}.{format}'

In [12]:
ALL_TISSUES = GTEXModel.get_tissues(conf.GTEX_MODELS_DIR)
assert len(ALL_TISSUES) == 49

### Compute results

In [13]:
# import threading
# from queue import Queue

from utils import simplify_string_for_hdf5

In [14]:
# to_write = Queue(maxsize=5)

In [15]:
def _get_combined_results(phenos, tissue, column):
    return {
        pheno.pheno_info.get_plain_name():
            pheno.get_tissue_data(tissue, cols=[column], index_col='gene_simple')
        for pheno in phenos
    }

In [16]:
# def dataframe_creator(pqueue):
def dataframe_creator(results, tissue_name, column_name):
#     for results, tissue_name, column_name in iter(pqueue.get, None):
    log_prefix = f'[{tissue_name} - {column_name}]'

    _n_expected_phenos = np.sum(list(conf.SMULTIXCAN_EXPECTED_PHENOTYPES.values()))
    assert len(results) == _n_expected_phenos, len(results)

    spredixcan = pd.DataFrame(results)
    spredixcan.index.rename('gene_name', inplace=True)
    assert spredixcan.index.is_unique

    # hdf5
    spredixcan_filename = OUTPUT_FILE_FORMAT.format(
        tissue=tissue_name,
        column=column_name,
        format='h5'
    )
    spredixcan_filename = os.path.join(OUTPUT_FOLDER, spredixcan_filename)

    print(f'  {log_prefix}, saving to: {spredixcan_filename}', flush=True)
    with pd.HDFStore(spredixcan_filename, mode='w', complevel=1) as store:
        for col in spredixcan.columns:
            clean_col = simplify_string_for_hdf5(col)
            store[clean_col] = spredixcan[col]#.astype(float)

In [17]:
def _run(tissue, column, phenotype_chunks, n_jobs=conf.N_JOBS_HIGH):
    all_results = {}
    with ProcessPoolExecutor(max_workers=n_jobs) as executor:
        tasks = [executor.submit(_get_combined_results, chunk, tissue, column) for chunk in phenotype_chunks]
        for future in as_completed(tasks):
            res = future.result()
            all_results.update(res)

    dataframe_creator(all_results, tissue, column)
#     to_write.put((all_results, tissue, column))

In [18]:
def run_all(tissues, phenotype_chunks, n_jobs=conf.N_JOBS_HIGH):
    for tissue in tissues:
        print(tissue, flush=True)
        
        for column in ('pvalue', 'zscore', 'effect_size'):
            _run(tissue, column, phenotype_chunks, n_jobs)
    
#     to_write.put(None)

In [19]:
# phenotype_chunks = chunker(all_spredixcan_phenotypes[:5] + all_extra_phenotypes[:5], 2)
phenotype_chunks = chunker(all_spredixcan_phenotypes + all_extra_phenotypes, 25)

In [20]:
phenotype_chunks = list(phenotype_chunks)

In [21]:
tissues = GTEXModel.get_tissues(conf.GTEX_MODELS_DIR)
#tissues = tissues[:2]

In [22]:
# writing_thread = threading.Thread(target=dataframe_creator, args=((to_write),))
# writing_thread.start()

In [23]:
run_all(tissues, phenotype_chunks)

Thyroid


  [Thyroid - pvalue], saving to: /mnt/phenomexcan_base/gene_assoc/spredixcan/spredixcan-Thyroid-pvalue.h5


  [Thyroid - zscore], saving to: /mnt/phenomexcan_base/gene_assoc/spredixcan/spredixcan-Thyroid-zscore.h5


  [Thyroid - effect_size], saving to: /mnt/phenomexcan_base/gene_assoc/spredixcan/spredixcan-Thyroid-effect_size.h5


Skin_Not_Sun_Exposed_Suprapubic


...

  [Brain_Hypothalamus - effect_size], saving to: /mnt/phenomexcan_base/gene_assoc/spredixcan/spredixcan-Brain_Hypothalamus-effect_size.h5


Esophagus_Gastroesophageal_Junction


  [Esophagus_Gastroesophageal_Junction - pvalue], saving to: /mnt/phenomexcan_base/gene_assoc/spredixcan/spredixcan-Esophagus_Gastroesophageal_Junction-pvalue.h5


  [Esophagus_Gastroesophageal_Junction - zscore], saving to: /mnt/phenomexcan_base/gene_assoc/spredixcan/spredixcan-Esophagus_Gastroesophageal_Junction-zscore.h5


  [Esophagus_Gastroesophageal_Junction - effect_size], saving to: /mnt/phenomexcan_base/gene_assoc/spredixcan/spredixcan-Esophagus_Gastroesophageal_Junction-effect_size.h5


In [24]:
# writing_thread.join()

## Testing

In [25]:
n_expected_phenos = np.sum(list(conf.SMULTIXCAN_EXPECTED_PHENOTYPES.values()))

In [26]:
n_expected_phenos

4091

In [27]:
output_hdf5_file = os.path.join(OUTPUT_FOLDER, 'spredixcan-Thyroid-pvalue.h5')

with pd.HDFStore(output_hdf5_file, mode='r') as store:
    store_keys = list(store.keys())
    assert len(store_keys) == n_expected_phenos
    display(store_keys[:5])
    
    clean_col = simplify_string_for_hdf5('N02-Diagnoses_main_ICD10_N02_Recurrent_and_persistent_haematuria')
    data = store[clean_col]
    assert data.shape == (15289,), data.shape
    assert data.loc['ENSG00000213965'] == 0.00023756504804916094, data.loc['ENSG00000213965']
    assert pd.isnull(data.loc['ENSG00000198670'])
    assert data.loc['ENSG00000177025'] == 1.586957013502016e-05, data.loc['ENSG00000177025']
    
    clean_col = simplify_string_for_hdf5('MAGNETIC_LDL.C')
    data = store[clean_col]
    assert data.shape == (15289,), data.shape
    assert data.loc['ENSG00000113163'] == 1.3600016190892495e-13, data.loc['ENSG00000113163']
    assert pd.isnull(data.loc['ENSG00000223510'])
    assert data.loc['ENSG00000204241'] == 0.9990521310244208, data.loc['ENSG00000204241']

['/c100001_raw_Food_weight',
 '/c100002_raw_Energy',
 '/c100003_raw_Protein',
 '/c100004_raw_Fat',
 '/c100005_raw_Carbohydrate']

In [28]:
output_hdf5_file = os.path.join(OUTPUT_FOLDER, 'spredixcan-Thyroid-zscore.h5')

with pd.HDFStore(output_hdf5_file, mode='r') as store:
    store_keys = list(store.keys())
    assert len(store_keys) == n_expected_phenos
    display(store_keys[:5])
    
    clean_col = simplify_string_for_hdf5('N02-Diagnoses_main_ICD10_N02_Recurrent_and_persistent_haematuria')
    data = store[clean_col]
    assert data.shape == (15289,), data.shape
    assert data.loc['ENSG00000213965'] == -3.6753054157625686, data.loc['ENSG00000213965']
    assert pd.isnull(data.loc['ENSG00000198670'])
    assert data.loc['ENSG00000177025'] == 4.316259089446458, data.loc['ENSG00000177025']
    
    clean_col = simplify_string_for_hdf5('MAGNETIC_LDL.C')
    data = store[clean_col]
    assert data.shape == (15289,), data.shape
    assert data.loc['ENSG00000113163'] == -7.400179862976074, data.loc['ENSG00000113163']
    assert pd.isnull(data.loc['ENSG00000223510'])
    assert data.loc['ENSG00000204241'] == 0.0011879778668467532, data.loc['ENSG00000204241']

['/c100001_raw_Food_weight',
 '/c100002_raw_Energy',
 '/c100003_raw_Protein',
 '/c100004_raw_Fat',
 '/c100005_raw_Carbohydrate']

In [29]:
output_hdf5_file = os.path.join(OUTPUT_FOLDER, 'spredixcan-Thyroid-effect_size.h5')

with pd.HDFStore(output_hdf5_file, mode='r') as store:
    store_keys = list(store.keys())
    assert len(store_keys) == n_expected_phenos
    display(store_keys[:5])
    
    clean_col = simplify_string_for_hdf5('N02-Diagnoses_main_ICD10_N02_Recurrent_and_persistent_haematuria')
    data = store[clean_col]
    assert data.shape == (15289,), data.shape
    assert data.loc['ENSG00000213965'] == -0.0011122695712738851, data.loc['ENSG00000213965']
    assert pd.isnull(data.loc['ENSG00000198670'])
    assert data.loc['ENSG00000177025'] == 0.0013606910719667048, data.loc['ENSG00000177025']
    
    clean_col = simplify_string_for_hdf5('MAGNETIC_LDL.C')
    data = store[clean_col]
    assert data.shape == (15289,), data.shape
    assert pd.isnull(data.loc['ENSG00000113163'])
    assert pd.isnull(data.loc['ENSG00000223510'])
    assert pd.isnull(data.loc['ENSG00000204241'])

['/c100001_raw_Food_weight',
 '/c100002_raw_Energy',
 '/c100003_raw_Protein',
 '/c100004_raw_Fat',
 '/c100005_raw_Carbohydrate']

In [30]:
output_hdf5_file = os.path.join(OUTPUT_FOLDER, 'spredixcan-Skin_Not_Sun_Exposed_Suprapubic-pvalue.h5')

with pd.HDFStore(output_hdf5_file, mode='r') as store:
    store_keys = list(store.keys())
    assert len(store_keys) == n_expected_phenos
    display(store_keys[:5])
    
    clean_col = simplify_string_for_hdf5('N02-Diagnoses_main_ICD10_N02_Recurrent_and_persistent_haematuria')
    data = store[clean_col]
    assert data.shape == (14920,), data.shape
    assert data.loc['ENSG00000214575'] == 0.999887282076106, data.loc['ENSG00000214575']
    assert pd.isnull(data.loc['ENSG00000231131'])
    assert data.loc['ENSG00000177025'] == 2.0403800371097046e-05, data.loc['ENSG00000177025']

['/c100001_raw_Food_weight',
 '/c100002_raw_Energy',
 '/c100003_raw_Protein',
 '/c100004_raw_Fat',
 '/c100005_raw_Carbohydrate']

The code below was used to write the assert above; see for each gene if first and last (min and max) correspond to sign above

In [31]:
rapid_gwas_dir = conf.SPREDIXCAN_RESULTS_DIR['RapidGWASProject']
gtex_gwas_dir = conf.SPREDIXCAN_RESULTS_DIR['GTEX_GWAS']

In [32]:
%%bash -s "$rapid_gwas_dir"
cd $1/N02
head -1 N02-gtex_v8-Thyroid-2018_10.csv | cut -f1-5 -d, | column -s, -t
echo ""
cat N02-gtex_v8-Thyroid-2018_10.csv | grep ENSG00000213965 | cut -f1-5 -d, | column -s, -t
echo ""
cat N02-gtex_v8-Thyroid-2018_10.csv | grep ENSG00000198670 | cut -f1-5 -d, | column -s, -t
echo ""
cat N02-gtex_v8-Thyroid-2018_10.csv | grep ENSG00000177025 | cut -f1-5 -d, | column -s, -t

gene  gene_name  zscore  effect_size  pvalue

ENSG00000213965.3  NUDT19  -3.675305415762569  -0.0011122695712738851  0.00023756504804916094

ENSG00000198670.11  LPA  NA  NA  NA

ENSG00000177025.3  C19orf18  4.316259089446458  0.0013606910719667048  1.586957013502016e-05


In [33]:
%%bash -s "$rapid_gwas_dir"
cd $1/N02
head -1 N02-gtex_v8-Skin_Not_Sun_Exposed_Suprapubic-2018_10.csv | cut -f1-5 -d, | column -s, -t
echo ""
cat N02-gtex_v8-Skin_Not_Sun_Exposed_Suprapubic-2018_10.csv | grep ENSG00000214575 | cut -f1-5 -d, | column -s, -t
echo ""
cat N02-gtex_v8-Skin_Not_Sun_Exposed_Suprapubic-2018_10.csv | grep ENSG00000231131 | cut -f1-5 -d, | column -s, -t
echo ""
cat N02-gtex_v8-Skin_Not_Sun_Exposed_Suprapubic-2018_10.csv | grep ENSG00000177025 | cut -f1-5 -d, | column -s, -t

gene  gene_name  zscore  effect_size  pvalue

ENSG00000214575.9  CPEB1  -0.00014127096801530567  -1.8697265305035635e-07  0.9998872820761059

ENSG00000231131.6  LINC01468  NA  NA  NA

ENSG00000177025.3  C19orf18  4.260425591320433  0.0009033853136217764  2.0403800371097043e-05


In [34]:
%%bash -s "$gtex_gwas_dir"
cd $1/MAGNETIC_LDL.C
head -1 spredixcan_igwas_gtexmashrv8_MAGNETIC_LDL.C__PM__Thyroid.csv | cut -f1-5 -d, | column -s, -t
echo ""
cat spredixcan_igwas_gtexmashrv8_MAGNETIC_LDL.C__PM__Thyroid.csv | grep ENSG00000113163 | cut -f1-5 -d, | column -s, -t
echo ""
cat spredixcan_igwas_gtexmashrv8_MAGNETIC_LDL.C__PM__Thyroid.csv | grep ENSG00000223510 | cut -f1-5 -d, | column -s, -t
echo ""
cat spredixcan_igwas_gtexmashrv8_MAGNETIC_LDL.C__PM__Thyroid.csv | grep ENSG00000204241 | cut -f1-5 -d, | column -s, -t

gene  gene_name  zscore  effect_size  pvalue

ENSG00000113163.15  COL4A3BP  -7.400179862976074  NA  1.3600016190892497e-13

ENSG00000223510.6  CDRT15  NA  NA  NA

ENSG00000204241.7  RP11-713P17.3  0.0011879778668467532  NA  0.9990521310244207
